In [ ]:
import re
import numpy as np
import matplotlib.pyplot as plt
import copy
import open3d as o3d

In [ ]:
def get_grid(s):
    pattern = re.compile(r'user_message\|([0-9\.]*)')
    grid = []
    for i in re.findall(pattern, s):
        if i == "":
            continue
        i = i.split('.')
        grid.append([int(x) for x in i])
    grid = np.array(grid)
    return grid

In [ ]:
def get_points(s):
    pattern = re.compile(r'user_message\|([0-9\.\,\-e]*,)')
    new_points = []
    for i in re.findall(pattern, s):
        i = i.split(',')[:-1]
        if len(i) == 0:
            continue
        new_points.append([float(x) for x in i])
    new_points = np.array(new_points)
    return new_points

In [ ]:
def visualize_pointcloud(pcd, show_draw=False):
    pcd = copy.deepcopy(pcd)
    o3d.visualization.draw_plotly([pcd])
    if show_draw:
        R = pcd.get_rotation_matrix_from_xyz((-np.pi / 2, 0, np.pi / 2))
        pcd.rotate(R, center=(0, 0, 0))
        o3d.visualization.draw([pcd], show_ui=True)

In [ ]:
def max_walls(grid):
    u, count = np.unique(grid, return_counts=True)

    count_sort_ind = np.argsort(-count)

    if len(count_sort_ind) <= 1:
        return
    grid[grid == u[count_sort_ind[1]]] = grid.max()

# Occupancy Map

In [ ]:
s = input()

In [ ]:
def draw_circle(radius):
    diameter = 2 * radius
    diameter += 10
    circle = [[0 for _ in range(diameter + 1)] for _ in range(diameter + 1)]

    x = radius
    y = 0
    error = 0

    while x >= y:
        circle[radius + x][radius + y] = 1
        circle[radius + y][radius + x] = 1
        circle[radius - y][radius + x] = 1
        circle[radius - x][radius + y] = 1
        circle[radius - x][radius - y] = 1
        circle[radius - y][radius - x] = 1
        circle[radius + y][radius - x] = 1
        circle[radius + x][radius - y] = 1

        y += 1
        error += 1 + 2 * y
        if 2 * (error - x) + 1 > 0:
            x -= 1
            error += 1 - 2 * x

    return circle

def print_circle(circle):
    for row in circle:
        print(' '.join(map(str, row)))

radius = 20
circle = draw_circle(radius)


In [ ]:
grid = np.array(circle)
max_walls(grid)

fig, ax = plt.subplots()

ax.imshow(grid, cmap='Greys')
ax.set_xticks([])
ax.set_yticks([])

plt.show()

# Lidar Point Cloud

In [ ]:
with open('../../data/logs/Drone1-HighLevelPathPlanning-2-25-2024.log') as f:
    s = f.read()

In [ ]:
s = input()

In [ ]:
points = get_points(s)
#points[:, 1] *= -1

pcl = o3d.geometry.PointCloud()
pcl.points = o3d.utility.Vector3dVector(points)

visualize_pointcloud(pcl)

# Create Messages

In [ ]:
import json

In [ ]:
def JSON_to_Dictonary(filename):
    with open(filename) as f:
        data = json.load(f)

        # get first timestamp
        if 'point_cloud' not in data.keys():
            data = next(iter(data.values()))

    return data

In [ ]:
data = JSON_to_Dictonary('../lidar/lidar_data_sample_boxed_env.json')

In [ ]:
from datetime import datetime

def log_message(message):
    with open('.output.txt', 'a') as f:
        f.write("{}|{}|{}|{}\n".format(
                                datetime.utcnow(),
                                "Drone1",
                                "user_message",
                                message
                            )
                        )

In [ ]:
import os
os.remove(".output.txt") 

points = data['point_cloud']
for r in range(len(points)):
    string = ''
    for c in range(len(points[0])):
        string += str(points[r][c]) + ","
    log_message(string)